In [1]:
%matplotlib inline

import pandas as pd
import intake
catalog = intake.cat.access_nri
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from mpl_toolkits.basemap import Basemap # plot on map projections
import netCDF4 as nc
import IPython.display
import cmocean
from glob import glob
import os,sys
import yaml
import math
import xarray as xr
import copy


os.chdir(os.path.join(os.getcwd(), "ACCESS-OM2-updated/figures/bathymetry"))
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])

Available exptdata keys:  ['1deg', '025deg', '01deg']


In [2]:
figdir = ''
def savefigure(fname):
    plt.savefig(os.path.join(figdir, fname),dpi=300, bbox_inches="tight")  # comment out to disable saving
#     plt.savefig(os.path.join(figdir, fname+'.pdf'),dpi=300, bbox_inches="tight")  # comment out to disable saving
    return
fileprefix = ''

In [3]:
data = copy.deepcopy(exptdata.exptdict)  # to store fields under the same keys as exptdata.exptdict

# inputs used for final runs, copied from /short/public
data['1deg']['mominput'] = 'access-om2/input_236a3011/mom_1deg'
data['025deg']['mominput'] = 'access-om2/input_236a3011/mom_025deg'
data['01deg']['mominput'] = 'access-om2/input_38570c62/mom_01deg'

In [ ]:
# add new ryf run
data['01deg_jra55v13_ryf9091'] = \
              {'model': 'access-om2-01',
               'expt': '01deg_jra55v13_ryf9091/',
               'desc': 'ACCESS-OM2-01 RYF 1990-91',
               'n_files': None,
#                'time_units': 'days since 0001-01-01',
               'time_units': 'days since 1988-01-01', # so .sel(time=slice(pd.Timestamp(1998,1,1),None)) removes first 10 years
               'offset': None,
               'exptdir': '/g/data3/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_ryf9091/',
               'mominput': 'access-om2/input_08022019/mom_01deg'}
fileprefix = 'new_'

In [ ]:
# add new ryf run
data['01deg_jra55v13_ryf9091_topo_bug_test'] = \
              {'model': 'access-om2-01',
               'expt': '01deg_jra55v13_ryf9091_topo_bug_test/',
               'desc': 'ACCESS-OM2-01 RYF 1990-91 topo bug test',
               'n_files': None,
#                'time_units': 'days since 0001-01-01',
               'time_units': 'days since 1988-01-01', # so .sel(time=slice(pd.Timestamp(1998,1,1),None)) removes first 10 years
               'offset': None,
               'exptdir': '/g/data3/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_ryf9091_topo_bug_test/',
               'mominput': 'access-om2/input_08022019/mom_01deg'}
fileprefix = 'new_'

In [4]:
for ekey in data.keys():
    e = data[ekey]
    outdirs = glob(os.path.join(e['exptdir'], 'output*'))
    outdirs.sort()
    e['lastout'] = outdirs[-1]
    print(ekey, e['lastout'])
    e['st_edges_ocean'] = xr.open_dataset(e['lastout']+'/ocean/ocean.nc').st_edges_ocean#.values
    e['kmt'] = xr.open_dataset(e['lastout']+'/ocean/ocean_grid.nc').kmt #.values
    e['depth'] = xr.open_dataset(e['mominput']+'/topog.nc').depth  # partial depth
    kmt_idx = e['kmt'].astype(np.int64).clip(min=0)
    tmp = e['st_edges_ocean'][kmt_idx]  # full depth, land is zero
    e['depth_full'] = tmp.where(tmp > 0)  # full depth, land is nan
    dzfull = e['st_edges_ocean'].diff('st_edges_ocean')
    dzftmp = dzfull[e['kmt'].astype(np.int64).clip(min=1)-1]  # full depth dz, land is wrong
    e['dz_full'] = dzftmp.where(tmp > 0)  # full depth dz, land is nan
    dztmp = e['depth'].values - e['st_edges_ocean'][(kmt_idx-1).clip(min=0)]  # partial depth dz, land is wrong
    e['dz'] = dztmp.where(tmp > 0)  # partial depth dz, land is nan

1deg /g/data/hh5/tmp/cosima/access-om2/1deg_jra55v13_iaf_spinup1_B1/output059


FileNotFoundError: [Errno 2] No such file or directory: b'/home/198/mp7041/ACCESS-OM2-updated/figures/bathymetry/access-om2/input_236a3011/mom_1deg/topog.nc'

In [ ]:
for ekey in data.keys():
    e = data[ekey]
    print(ekey, e['desc'])
    print('min levels =', int(e['kmt'].min().item(0)))
    print('min partial cell depth   =', e['depth'].where(e['depth']>0).min().item(0),'m')
#     print('min full cell depth =', e['st_edges_ocean'][int(e['kmt'].min().item(0))].item(0),'m')
    print('min full cell depth =', e['depth_full'].min().item(0),'m')
    print('max full cell depth =', e['depth_full'].max().item(0),'m')
    print('min partial cell thickness at bottom =', e['dz'].min().item(0),'m')
    print('min full cell thickness at bottom =', e['dz_full'].min().item(0),'m')
    ratio = e['dz']/e['dz_full']
    print('min partial/full thickness at bottom =', ratio.min().item(0))
    print('max partial/full thickness at bottom =', ratio.max().item(0))
    print('number of cells with partial > full thickness at bottom =', ratio.where(ratio>1).count().item(0))
    print('min partial-full thickness at bottom, where partial>full =', (e['dz']-e['dz_full']).where(e['dz']>e['dz_full']+1e-5).min().item(0))
    print('max partial-full thickness at bottom =', (e['dz']-e['dz_full']).max().item(0))

    print()

In [ ]:
e['depth_full'].plot()

In [ ]:
e['dz_full'].plot()

In [ ]:
e['dz'].plot()

In [ ]:
ratio.plot()

In [ ]:
ratio.where(ratio>-1).plot(figsize=(12,6))

In [ ]:
thickdiff = e['dz']-e['dz_full']
thickdiff.where(thickdiff>=0).plot(figsize=(12,6))

In [ ]:
# depth below which there is some degree of partial cell
data['01deg']['st_edges_ocean'][1:].where(e['st_edges_ocean'].diff('st_edges_ocean')>10)

In [ ]:
# depth below which 20% threshold is actual lower limit
data['01deg']['st_edges_ocean'][1:].where(e['st_edges_ocean'].diff('st_edges_ocean')>50)

In [ ]:
plt.figure(figsize=(12,6))
for i, ekey in enumerate(data.keys()):
    e = data[ekey]
    plt.subplot(1,len(data.keys()),i+1)
    plt.plot([0, 230],[0,230],'k',linewidth=0.2)
    plt.plot([0, 230],[0,230*0.2],'k',linewidth=0.2)
    plt.scatter(e['dz_full'], e['dz'], s=0.2)
    plt.xlim(0,230)
    plt.ylim(0,230)
    plt.yticks(np.arange(0, 231, step=10))
    plt.xlabel('Full cell dz (m)')
    plt.title(e['desc'])
    if i == 0:
        plt.ylabel('Partial cell dz (m)')
    else:
        plt.gca().axes.yaxis.set_ticklabels([])
plt.tight_layout()
savefigure(fileprefix+'partialcells.png')

In [ ]:
plt.figure(figsize=(12,6))
for i, ekey in enumerate(data.keys()):
    e = data[ekey]
    plt.subplot(1,len(data.keys()),i+1)
    plt.plot([0, 20],[0,20],'k',linewidth=0.2)
    plt.plot([0, 20],[0,20*0.2],'k',linewidth=0.2)
    plt.scatter(e['dz_full'], e['dz'], s=0.2)
    plt.xlim(0,20)
    plt.ylim(0,20)
    plt.yticks(np.arange(0, 21, step=1))
    plt.xlabel('Full cell dz (m)')
    plt.title(e['desc'])
    if i == 0:
        plt.ylabel('Partial cell dz (m)')
    else:
        plt.gca().axes.yaxis.set_ticklabels([])
plt.tight_layout()
savefigure(fileprefix+'partialcells_closeup.png')

In [ ]:
plt.figure(figsize=(12,6))
for ekey in data.keys():
    e = data[ekey]
    plt.hist(np.ma.ravel(e['depth']), bins=20000, alpha=0.5, log=True, label=e['desc'])
plt.xlim(10,6000)
plt.xscale('log')
plt.xlabel('Depth (m)')
plt.ylabel('Count');
plt.title('Histogram of model depth');
plt.legend(loc='upper right')

In [ ]:
plt.figure(figsize=(12,6))
for ekey in data.keys():
    e = data[ekey]
    plt.hist(np.ma.ravel(e['dz']-e['dz_full']), bins=20000, alpha=0.5, log=True, label=e['desc'])
plt.xlim(0.001,2)
plt.xlabel('Partial depth - full depth (m)')
plt.ylabel('Count');
plt.title('Histogram of partial minus full depth');
plt.legend(loc='upper right')

In [ ]:
e['dz'].plot()

In [ ]:
e['dz_full'].plot()

In [ ]:
e['depth_full'].plot()

In [ ]:
e['depth'].plot()